In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.2/224.2 kB 8.1 MB/s eta 0:00:00


In [ ]:
# Import Pytorch, AutoModel and AutoTokenizer
import torch
from transformers import AutoModel, AutoTokenizer

# Choose a pre-trained model
model = AutoModel.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# Define the sentences to be tokenized
sentences = ["I love my family", "I love my dog", "My dog is a lab"]

In [ ]:
# Create tokens from the tokenizer instance.
tokens = [tokenizer.tokenize(sentence) for sentence in sentences]

# Print out the tokens.
print(tokens)

[['I', 'love', 'my', 'family'], ['I', 'love', 'my', 'dog'], ['My', 'dog', 'is', 'a', 'lab']]


In [ ]:
# Convert the tokens to IDs.
tokenized_sentences = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
input_ids = tokenized_sentences['input_ids']
print(input_ids)

tensor([[ 101,  146, 1567, 1139, 1266,  102,    0],
        [ 101,  146, 1567, 1139, 3676,  102,    0],
        [ 101, 1422, 3676, 1110,  170, 8074,  102]])


In [ ]:
# Generate embeddings using PyTorch
with torch.no_grad():
    outputs = model(tokenized_sentences['input_ids'])
    embeddings = outputs.last_hidden_state[:, 0, :].numpy()

# Print the embeddings
for i, sentence in enumerate(sentences):
    print(f"Sentence {i+1}: {sentence}")
    print(f"Embedding {i+1}: {embeddings[i][0:10]}")
    print()

Sentence 1: I love my family
Embedding 1: [-0.19375661  0.32130736  0.07548827 -0.40688723 -0.4763685   0.07858068
  0.8525571   0.05237346 -0.06373273 -0.1726824 ]

Sentence 2: I love my dog
Embedding 2: [-0.18957779  0.6149509   0.42388856 -0.31033877 -0.55150765 -0.04185519
  0.68279636  0.19014312 -0.2128999  -0.1065003 ]

Sentence 3: My dog is a lab
Embedding 3: [-0.37911138  0.05389749 -0.32136977 -0.34749442 -0.16758227 -0.08009627
  0.09228063  0.4515096  -0.44420296 -0.25733727]



In [ ]:
# Get the length of each embedding.
[print(len(embedding)) for embedding in embeddings]

768
768
768


[None, None, None]

In [ ]:
# Import numpy and the cosine function from scipy.spatial.distance 
import numpy as np
from scipy.spatial.distance import cosine


# Create a 3x3 matrix with zeros. 
similarity_matrix = np.zeros((len(embeddings), len(embeddings)))
# First iteration through each embedding 
for i in range(len(embeddings)):
  # Second iteration through each embedding.
  for j in range(len(embeddings)):
    # Calculate the pairwise cosine similarities between the embeddings.
    similarity_matrix[i, j] = 1 - cosine(embeddings[i], embeddings[j])

# Print the similarity matrix
print("Similarity matrix:")
print(similarity_matrix)

Similarity matrix:
[[1.         0.96456212 0.82049882]
 [0.96456212 1.         0.8291617 ]
 [0.82049882 0.8291617  1.        ]]


In [ ]:
# Import Pandas
import pandas as pd

# Convert the similarity matrix to a Pandas DataFrame. 
similarity_df = pd.DataFrame(similarity_matrix, columns=['Sentence 1', 'Sentence 2', 'Sentence 3'], index=['Sentence 1', 'Sentence 2', 'Sentence 3'])
similarity_df

,Sentence 1,Sentence 2,Sentence 3
Sentence 1,1.000000,0.964562,0.820499
Sentence 2,0.964562,1.000000,0.829162
Sentence 3,0.820499,0.829162,1.000000
